In [1]:
cd ..

/home/scott/Documents/git/bite-me


In [2]:
import sklearn
import pandas as pd
import numpy as np
import datetime

In [3]:
from IPython.display import HTML
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

/usr/local/lib/python3.4/dist-packages/matplotlib/backends/backend_gtk3agg.py:18: UserWarning: The Gtk3Agg backend is known to not work on Python 3.x with pycairo. Try installing cairocffi.
  "The Gtk3Agg backend is known to not work on Python 3.x with pycairo. "


In [4]:
import sklearn.linear_model
import sklearn.cross_validation

In [6]:
from eat_it import StratifiedPercentileKFold
from eat_it import boxcoxscaler

In [7]:
import imp

In [8]:
train = pd.read_csv('data/train.csv', encoding="utf-8")

In [9]:
# Add age in days
end_dt = datetime.datetime.strptime('2015-1-1', "%Y-%m-%d")
train['Age'] = [(end_dt - datetime.datetime.strptime(open_dt, "%m/%d/%Y")).days for open_dt in train['Open Date']]

In [10]:
# add size as boolean
train['isBig'] = train['City Group']=='Big Cities'

In [11]:
train['isIL'] = train['Type']=='IL'

In [12]:
cols = ['P'+str(i) for i in range(1,38)]
cols.append('Age')
cols.append('isBig')
cols.append('isIL')

In [13]:
cols

['P1',
 'P2',
 'P3',
 'P4',
 'P5',
 'P6',
 'P7',
 'P8',
 'P9',
 'P10',
 'P11',
 'P12',
 'P13',
 'P14',
 'P15',
 'P16',
 'P17',
 'P18',
 'P19',
 'P20',
 'P21',
 'P22',
 'P23',
 'P24',
 'P25',
 'P26',
 'P27',
 'P28',
 'P29',
 'P30',
 'P31',
 'P32',
 'P33',
 'P34',
 'P35',
 'P36',
 'P37',
 'Age',
 'isBig',
 'isIL']

In [14]:
X = train.as_matrix(cols)

In [15]:
X = X.astype(np.float)

In [17]:
y = train['revenue'].values

In [18]:
X_indices = train['Id'].values

In [19]:
def do_cv(cv, clf, y_transform=None, X=X, y=y):
    if y_transform is None:
        y_transform = lambda x: x
    scores = []
    all_pred = np.zeros((y.shape))
    for train_index, test_index in cv:
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        # Fit model
        clf.fit(X_train, y_train)
        # Predict scores for test data
        y_pred = y_transform(clf.predict(X_test))
        # Save all the predictions to an array
        all_pred[test_index] = y_pred
        # Compute mean squared error on this test set
        mse = sklearn.metrics.mean_squared_error(y_transform(y_test),y_pred)
        scores.append(mse)
    # Compute MSE across all samples
    all_score = sklearn.metrics.mean_squared_error(y_transform(y), all_pred)**0.5
    scores = np.asarray(scores)**0.5
    return all_score, scores

In [21]:
def get_cv_results(clf, X=X, y=y, n_folds=10, n_samp=25):
    all_scores = []
    for seed in range(n_samp):
        cv = StratifiedPercentileKFold.StratifiedPercentileKFold(y, n_folds=n_folds, shuffle=True, random_state=seed, shuffle_windows=True)
        this_score, _ = do_cv(cv, clf, X=X, y=y)
        all_scores.append(this_score)
    return np.mean(all_scores), np.std(all_scores)/np.sqrt(n_folds)

def get_mean_cv_score(*args, **kwargs):
    out = get_cv_results(*args, **kwargs)
    return out[0]

In [44]:
from sklearn.base import clone

In [58]:
### Recursive feature addition

def rfa(clf, X, y, n_folds=10, n_samp=25, col_names=None):

    n_features = X.shape[1]
    n_features_to_select = n_features
    step = 1
    
    if col_names is None:
        col_names = range(n_features)
    col_names = np.asarray(col_names)
    
    support_ = np.zeros(n_features, dtype=np.bool)
    ranking_ = n_features * np.ones(n_features, dtype=np.int)
    last_score = None

    # Feature addition
    while np.sum(support_) < n_features_to_select:
        # Previously added features
        features_already = np.arange(n_features)[support_]
        # Features to test
        features_to_test = np.arange(n_features)[np.logical_not(support_)]

        # Rank the remaining features
        estimator = clone(clf)

        #####################################
        # FIT THE CLASSIFIER ON A NESTED FOLD
        #####################################

        scores = np.zeros(len(features_to_test))
        for feature_index, test_feature in enumerate(features_to_test):
            features = np.union1d(features_already, [test_feature])
            scores[feature_index] = get_mean_cv_score(estimator, X[:, features], y, n_folds=n_folds, n_samp=n_samp)
            print("\tScored %.2f with %s" % (scores[feature_index], ', '.join(col_names[features])))

        # Sort the scores in ascending order
        score_order_index = np.argsort(scores)
        ordered_scores   = scores[score_order_index]
        ordered_features = features_to_test[score_order_index]

        # Break if no features can improve score
        if last_score is not None and last_score < ordered_scores[0]:
            print('No more improvement possible from {} to {} features'.format(
                    len(features_already),len(features_already)+1))
            break

        # Only add `step` many features if it doesn't take us past the target
        n_add = min(step, n_features_to_select - np.sum(support_))

        # Only add features which don't make performance go down
        if last_score is not None:
            n_add = min(n_add, len(np.nonzero(ordered_scores < last_score)))

        # Select best.
        # We will MINIMISE scoring function!!!
        features_to_add = ordered_features[0:n_add]
        for i in range(n_add):
            print('Adding feature {} (scored {})'.format(col_names[ordered_features[i]], ordered_scores[i]))

        # Add the features
        support_[features_to_add] = True
        ranking_[features_to_add] = np.sum(support_) + 1 + np.arange(features_to_add)

        # Update score monitor
        last_score = ordered_scores[0]
    
    print("Best score is with features:\n\t{}".format(', '.join(col_names[support_])))
    
    return support_, ranking_

In [57]:
clf = sklearn.linear_model.LinearRegression()
support, ranking = rfa(clf, X, y, col_names=cols)

	Scored 2576920.82 with P1
	Scored 2532828.62 with P2
	Scored 2580289.25 with P3
	Scored 2578094.47 with P4
	Scored 2575959.25 with P5
	Scored 2561503.10 with P6
	Scored 2575867.06 with P7
	Scored 2576849.75 with P8
	Scored 2578674.69 with P9
	Scored 2578549.73 with P10
	Scored 2572805.21 with P11
	Scored 2580951.29 with P12
	Scored 2575454.84 with P13
	Scored 2584633.04 with P14
	Scored 2586147.57 with P15
	Scored 2578817.19 with P16
	Scored 2600917.89 with P17
	Scored 2578976.84 with P18
	Scored 2578747.33 with P19
	Scored 2578007.73 with P20
	Scored 2582338.00 with P21
	Scored 2590895.79 with P22
	Scored 2574120.03 with P23
	Scored 2585463.56 with P24
	Scored 2585614.41 with P25
	Scored 2583715.51 with P26
	Scored 2578944.67 with P27
	Scored 2554509.59 with P28
	Scored 2579073.51 with P29
	Scored 2573358.53 with P30
	Scored 2577914.49 with P31
	Scored 2570878.71 with P32
	Scored 2580696.14 with P33
	Scored 2571085.48 with P34
	Scored 2578209.98 with P35
	Scored 2576505.42 with P36
	

In [59]:
import sklearn.svm
clf = sklearn.svm.SVR()
support, ranking = rfa(clf, X, y, col_names=cols)

	Scored 2618310.77 with P1
	Scored 2618309.70 with P2
	Scored 2618310.80 with P3
	Scored 2618310.56 with P4
	Scored 2618310.27 with P5
	Scored 2618308.95 with P6
	Scored 2618310.15 with P7
	Scored 2618310.63 with P8
	Scored 2618310.44 with P9
	Scored 2618309.97 with P10
	Scored 2618311.02 with P11
	Scored 2618310.33 with P12
	Scored 2618310.01 with P13
	Scored 2618310.06 with P14
	Scored 2618310.11 with P15
	Scored 2618309.87 with P16
	Scored 2618309.66 with P17
	Scored 2618309.86 with P18
	Scored 2618311.29 with P19
	Scored 2618310.19 with P20
	Scored 2618311.02 with P21
	Scored 2618311.27 with P22
	Scored 2618311.44 with P23
	Scored 2618310.10 with P24
	Scored 2618310.05 with P25
	Scored 2618310.16 with P26
	Scored 2618310.28 with P27
	Scored 2618310.55 with P28
	Scored 2618310.76 with P29
	Scored 2618309.91 with P30
	Scored 2618309.49 with P31
	Scored 2618309.96 with P32
	Scored 2618310.10 with P33
	Scored 2618310.05 with P34
	Scored 2618309.88 with P35
	Scored 2618309.78 with P36
	

In [63]:
import sklearn.ensemble
clf = sklearn.ensemble.RandomForestRegressor(250, random_state=42)
support, ranking = rfa(clf, X, y, col_names=cols)

	Scored 2612496.55 with P1
	Scored 2494475.23 with P2
	Scored 2634520.03 with P3
	Scored 2614852.42 with P4
	Scored 2554550.96 with P5
	Scored 2593298.22 with P6
	Scored 2563504.97 with P7
	Scored 2692234.54 with P8
	Scored 2602572.30 with P9
	Scored 2564310.39 with P10
	Scored 2629634.54 with P11
	Scored 2661862.37 with P12
	Scored 2564236.94 with P13
	Scored 2617744.18 with P14
	Scored 2630981.69 with P15
	Scored 2598948.03 with P16
	Scored 2512237.10 with P17


KeyboardInterrupt: 